我在 Tensorflow 官網的教學網頁上找到了用 Iris 當例題的 estimator tutorial https://www.tensorflow.org/versions/master/get_started/premade_estimators (also on my Ynote "Get Started with Estimators _TensorFlow_") 有正式、詳細的介紹。
這篇文章鼓勵人用 Estimator API 跟 Dataset API。本文研讀、實驗這篇文章。
文章引用的 GitHub resource 我已經 clone 下來了在 T550 `c:\Users\hcche\Documents\GitHub\models`  

重點是，直接用 pandas data-frame 來當作 feature - label 餵給 input function 這樣更自然，免去搞 150 天行情資料的麻煩。


In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
from sklearn import datasets
import numpy as np
import pandas as pd

In [1]:
import peforth
peforth.dictate(r'''
\ Now we redefine the 'unknown' command that was do-nothing by default  
: unknown ( token -- thing Y|N) // Try to find the unknown token in __main__
  py> getattr(sys.modules['__main__'],pop(),"Ûnknôwn") 
  py> str(tos())=="Ûnknôwn" if drop false else true then ;
  /// here after, when FORTH come accross an unknown token, instead of alerting 
  /// it try to find the token in python __main__ module name space.

\ Redefine \ command to print the comment line
code \ print(nexttoken('\n')) end-code ''');

reDef \


In [ ]:
iris = datasets.load_iris()

In [ ]:
iris.feature_names

In [ ]:
iris.data[:4]

In [ ]:
# df = pd.DataFrame(np.array(irisX).reshape(-1,4), columns = iris.feature_names)    
x = pd.DataFrame(iris.data, columns = iris.feature_names)    
x[:4]

In [ ]:
y = pd.DataFrame(iris.target, columns = ["target"])    
y[48:52]

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/estimator/inputs/pandas_input_fn

tf.estimator.inputs.pandas_input_fn(
    x,
    y,
    batch_size=64,
    num_epochs=5,
    shuffle=False,
    queue_capacity=None,
    num_threads=1,
    target_column='target'
)

# 照 Tutorial 執行 -- 一把就成功！

直接把本 notebook 建個 hard link 到 c:\Users\hcche\Documents\GitHub\models\samples\core\get_started 去執行...


In [ ]:
%run premade_estimator.py

## 開始照教材研讀
    
premade_estimator.py iris_data.py 都抓過來，他本來都放在 main() 內執行，不利 study 今予拆開，分段執行亦可，如下：

In [3]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
"""An Example of a DNNClassifier for the Iris dataset."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import tensorflow as tf

import iris_data  # iris_data.py 在本目錄下 GitHub\models\samples\core\get_started\iris_data.py 

這 argparse module 很常見 DeepSpeech 就有，這裡用不著吧！

In [4]:
parser = argparse.ArgumentParser()
parser.add_argument('--batch_size', default=100, type=int, help='batch size')
parser.add_argument('--train_steps', default=1000, type=int,
                    help='number of training steps')

_StoreAction(option_strings=['--train_steps'], dest='train_steps', nargs=None, const=None, default=1000, type=<class 'int'>, choices=None, help='number of training steps', metavar=None)

In [ ]:
# 不能在 jupyter notebook 下直接跑，因為 argv 不如預期。
from IPython.display import display,Image;display(Image('error1.jpg', width=700))

#### Command Line Arguments 要用模擬的
Module argparse 只能在 python command line 跑

In [5]:
%f \ [x] 沒有指定時，自動取 default 值，真不知道這哪來的？看到了，從 parser 那邊 code 裡給的。
args = parser.parse_args(["--batch_size=100", "--train_steps=1000"])
args

[x] 沒有指定時，自動取 default 值，真不知道這哪來的？看到了，從 parser 那邊 code 裡給的。


Namespace(batch_size=100, train_steps=1000)

#### 研究  (train_x, train_y), (test_x, test_y) = iris_data.load_data()

看出來了 iris_data.py 是把 150 筆資料切成 120 : 30 for training and testing respectively.
這是個壞消息，他沒有像 sklearn 的自動切的那啥 function. 

In [6]:
 (train_x, train_y), (test_x, test_y) = iris_data.load_data()

In [ ]:
%f train_x type . cr
train_x

In [ ]:
%f train_y type . cr
train_y

In [ ]:
%f test_x type . cr
test_x

In [ ]:
%f test_y type . cr
test_y

In [ ]:
%f \ [ ] 這是 feature columns, 我晚點可以嘗試把它弄成光一個 shape 為 (4,) 的單一 feature 
for key in train_x.keys():
    print(key)

In [7]:
%f \ [ ] 這是 feature columns, 我晚點可以嘗試把它弄成光一個 shape 為 (4,) 的單一 feature 
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
my_feature_columns    

[ ] 這是 feature columns, 我晚點可以嘗試把它弄成光一個 shape 為 (4,) 的單一 feature


[_NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [8]:
%f \ 從 main 裡拆分出來，方便 tracing. 可看出若無指定 model directory 自動放在
%f \     AppData\Local\Temp\tmpijrg8mkm <-- 每次都不一樣，這樣好嗎？
%f \ 以及本 classifier 的種種細節
%f \ 

# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 10 nodes each.
    hidden_units=[10, 10],
    # The model must choose between 3 classes.
    n_classes=3)

從 main 裡拆分出來，方便 tracing. 可看出若無指定 model directory 自動放在
    AppData\Local\Temp\tmpijrg8mkm <-- 每次都不一樣，這樣好嗎？
以及本 classifier 的種種細節

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\hcche\\AppData\\Local\\Temp\\tmpj_kgu6hi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000025E1B1D6C88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
%f \ 複習一下 lambda --> 
%f \ lambda 是個 function, 產生 key(i) 取 x 的第 i 個 element。用於 sorted() 時指定哪個當 key 
s = [('a', 3), ('b', 2), ('c', 1)]
key = lambda x:x[1]
print(sorted(s, key=key))
%f \ 單獨使用時，看起來不太一樣，其實做的是一樣的事情。
%f key :> ([('a',3),('b',2),('c',1),('d',4)]) tib.
%f key tib.
%f key .source
%f \ 當沒有給 lambda argument 時，是怎樣？
x = lambda:s
%f x tib. \ 直接傳回 : 之後的東西(執行後)的傳回值
%f x type tib.
%f \ 查看 x 的 source code 
%f x .source
%f x :> () tib.

%f \ 所以，input_fn = lambda:iris_data.train_input_fn(train_x, train_y,args.batch_size)
%f \ 當中的 iris_data.train_input_fn() 是傳回一個 function 但是該 function 的 argument 是活的，
%f \ 要延後到最後一秒鐘才確定，因此要用 lambda 或單純的 function 包一層用來指定 arguments。
    

In [9]:
%f \ 這就開始訓練了 . . .  
%f \ 

# Train the Model.
classifier.train(
    input_fn=lambda:iris_data.train_input_fn(train_x, train_y,
                                             args.batch_size),
    steps=args.train_steps)

這就開始訓練了 . . .

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\hcche\AppData\Local\Temp\tmpj_kgu6hi\model.ckpt.
INFO:tensorflow:loss = 190.65956, step = 1
INFO:tensorflow:global_step/sec: 341.14
INFO:tensorflow:loss = 11.143849, step = 101 (0.295 sec)
INFO:tensorflow:global_step/sec: 553.886
INFO:tensorflow:loss = 9.443028, step = 201 (0.180 sec)
INFO:tensorflow:global_step/sec: 533.364
INFO:tensorflow:loss = 7.366951, step = 301 (0.190 sec)
INFO:tensorflow:global_step/sec: 581.017
INFO:tensorflow:loss = 7.476582, step = 401 (0.171 sec)
INFO:tensorflow:global_step/sec: 543.432
INFO:tensorflow:loss = 9.928789, step = 501 (0.184 sec)
INFO:tensorflow:global_step/sec: 579.346
INFO:tensorflow:loss = 6.958041, step = 601 (0.173 sec)
INFO:tensorflow:glo

這時候的 classifier 已經受過訓練了! 可以接受考核了

In [10]:
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:iris_data.eval_input_fn(test_x, test_y,
                                            args.batch_size))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-05-26-11:50:40
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\hcche\AppData\Local\Temp\tmpj_kgu6hi\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-05-26-11:50:40
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.96666664, average_loss = 0.052972097, global_step = 1000, loss = 1.589163


In [11]:
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


Test set accuracy: 0.967



In [ ]:
%f \ 複習一下 **eval_result 啥意思？
%f eval_result . cr
%f eval_result :> ['accuracy'] . cr
{**eval_result}

In [ ]:
rslt = {'accuracy': 0.96666664, 'average_loss': 0.056607112, 'loss': 1.6982133, 'global_step': 1000}
print('Test set accuracy: {loss:0.3f}'.format(**rslt))
print('Test set accuracy: {:0.3f}'.format(r['accuracy']))
print('Test set accuracy: {a:0.3f}'.format(**{'a': 11,'b': 22}))
print('Test set accuracy: {b:0.3f}'.format(**{'a': 33,'b': 44}))
%f \ 最後一式拆解不成功，拆開來的寫法也沒意義，要嘛直接寫就好了。
%f \ print('Test set accuracy: {b:0.3f}'.format(({'a': 55},{'b': 66})))

In [23]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica','foo','bar']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
    'bar':[]
}

[(i,j) for i,j in zip(predict_x,expected)]
[i for i in zip(predict_x,expected)]


[('SepalLength', 'Setosa'),
 ('SepalWidth', 'Versicolor'),
 ('PetalLength', 'Virginica'),
 ('PetalWidth', 'foo'),
 ('bar', 'bar')]

In [37]:
predictions = classifier.predict(
    input_fn=lambda:iris_data.eval_input_fn(predict_x,
                                            labels=None,
                                            batch_size=args.batch_size))

In [38]:
%f \ 以上 predictions 瞬間就取得了
%f predictions tib.
%f \ 問題是這 'generator' 用過就沒了，怎麼看哪?
[i for i in predictions]

以上 predictions 瞬間就取得了
predictions tib. \ ==> <generator object Estimator.predict at 0x0000025E1CD380F8> (<class 'generator'>)
問題是這 'generator' 用過就沒了，怎麼看哪?


ValueError: Dimensions 3 and 0 are not compatible

In [31]:
template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

In [33]:
for pred_dict, expec in zip(predictions, expected):
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(iris_data.SPECIES[class_id],
                          100 * probability, expec))

In [ ]:
def main(argv):
    args = parser.parse_args("")  # default is (["--batch_size=100", "--train_steps=1000"])
    # Fetch the data
    (train_x, train_y), (test_x, test_y) = iris_data.load_data()

    # Feature columns describe how to use the input.
    my_feature_columns = []
    for key in train_x.keys():
        my_feature_columns.append(tf.feature_column.numeric_column(key=key))

    # Build 2 hidden layer DNN with 10, 10 units respectively.
    classifier = tf.estimator.DNNClassifier(
        feature_columns=my_feature_columns,
        # Two hidden layers of 10 nodes each.
        hidden_units=[10, 10],
        # The model must choose between 3 classes.
        n_classes=3)

    # Train the Model.
    classifier.train(
        input_fn=lambda:iris_data.train_input_fn(train_x, train_y,
                                                 args.batch_size),
        steps=args.train_steps)

    # Evaluate the model.
    eval_result = classifier.evaluate(
        input_fn=lambda:iris_data.eval_input_fn(test_x, test_y,
                                                args.batch_size))

    print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

    # Generate predictions from the model
    expected = ['Setosa', 'Versicolor', 'Virginica']
    predict_x = {
        'SepalLength': [5.1, 5.9, 6.9],
        'SepalWidth': [3.3, 3.0, 3.1],
        'PetalLength': [1.7, 4.2, 5.4],
        'PetalWidth': [0.5, 1.5, 2.1],
    }

    predictions = classifier.predict(
        input_fn=lambda:iris_data.eval_input_fn(predict_x,
                                                labels=None,
                                                batch_size=args.batch_size))

    template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

    for pred_dict, expec in zip(predictions, expected):
        class_id = pred_dict['class_ids'][0]
        probability = pred_dict['probabilities'][class_id]

        print(template.format(iris_data.SPECIES[class_id],
                              100 * probability, expec))


if __name__ == '__main__':
    tf.logging.set_verbosity(tf.logging.INFO)
    tf.app.run(main)

In [ ]:
import pandas as pd
import tensorflow as tf

TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

def maybe_download():
    train_path = tf.keras.utils.get_file(TRAIN_URL.split('/')[-1], TRAIN_URL)
    test_path = tf.keras.utils.get_file(TEST_URL.split('/')[-1], TEST_URL)

    return train_path, test_path

def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
    train_path, test_path = maybe_download()

    train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)

    test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y), (test_x, test_y)


def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset


def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset


# The remainder of this file contains a simple example of a csv parser,
#     implemented using the `Dataset` class.

# `tf.parse_csv` sets the types of the outputs to match the examples given in
#     the `record_defaults` argument.
CSV_TYPES = [[0.0], [0.0], [0.0], [0.0], [0]]

def _parse_line(line):
    # Decode the line into its fields
    fields = tf.decode_csv(line, record_defaults=CSV_TYPES)

    # Pack the result into a dictionary
    features = dict(zip(CSV_COLUMN_NAMES, fields))

    # Separate the label from the features
    label = features.pop('Species')

    return features, label


def csv_input_fn(csv_path, batch_size):
    # Create a dataset containing the text lines.
    dataset = tf.data.TextLineDataset(csv_path).skip(1)

    # Parse each line.
    dataset = dataset.map(_parse_line)

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset


    ----- play ground -----

In [ ]:
# This cell creates %%csv cell magic. See http://ipython-books.github.io/14-creating-an-ipython-extension-with-custom-magic-commands
import pandas as pd
from io import StringIO
from IPython.core.magic import (register_cell_magic)
@register_cell_magic
def csv(line, cell):
    return pd.read_csv(StringIO(cell), sep=line, engine='python')

In [ ]:
%%csv \s+
a b c
1 2 3
4 5 6
7 8 9